In [1]:
import cv2
import os
import random
import numpy as np

from register_dataset import *
from detectron2.engine import DefaultPredictor, default_setup
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo

from utils import draw_rect

In [2]:
dataset_name_train = "HRSIDtrain2017"
dataset_name_test = "HRSIDtest2017"
dataset_dicts = DatasetCatalog.get(dataset_name_test)
dataset_metadata = MetadataCatalog.get(dataset_name_test)

# Load model

In [3]:
def setup(config_file):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("Misc/mask_rcnn_R_50_FPN_3x_gn.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/mask_rcnn_R_50_FPN_3x_gn.yaml") 
    cfg.merge_from_file(config_file)
    default_setup(cfg, None)
    return cfg

In [4]:
# cfg already contains everything we've set previously. Now we changed it a little bit for inference:
cfg = setup(config_file='yamls/HRSID/MaskRCNN-R50.yaml' )
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1   # custom testing threshold for R-CNN
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.1   
predictor = DefaultPredictor(cfg)

[02/27 22:54:35 fvcore.common.checkpoint]: Loading checkpoint from output/HRSID/MaskRCNN-R50/model_final.pth


# Visualization

In [5]:
save_path = "../exp_figs/MaskRCNN-R50"
!mkdir -p $save_path

In [6]:
for i, d in enumerate(dataset_dicts):
    im = cv2.imread(d["file_name"])
    name = os.path.split(d["file_name"])[1]
    
    true_coords = [ann['bbox'] for ann in d['annotations']]
    true_coords = np.array(true_coords, dtype='int')  
    
    outputs = predictor(im)
    pred_coords = outputs['instances'].get_fields()['pred_boxes'].tensor.cpu().numpy().astype('int')
    pred_coords[:,2] -= pred_coords[:,0]
    pred_coords[:,3] -= pred_coords[:,1]    
    draw_rect(im, true_coords, pred_coords, save_path=os.path.join(save_path, str(i)))


/home/jtli/SAR/detectron2/detectron2/modeling/roi_heads/fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
